# Matchmaker Basic Usage

This notebook demonstrates the core matchmaker library functionality:
- Loading interaction data with gender support
- Computing engagement scores
- Computing popularity metrics and leagues
- Training ALS models for recommendations

### ⚠️ Note: NVIDIA Driver Installation Required ⚠️

Verify with: ```nvidia-smi```

In [1]:
from matchmaker import MatchingEngine
import cudf

/home/zain/anaconda3/envs/matchmaker-dev/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
raw = cudf.read_csv("data/swipes_clean.csv")

In [3]:
engine = MatchingEngine()

In [4]:
engine.load_interactions("data/swipes_clean.csv", 
  decider_col='decidermemberid',
  other_col='othermemberid', 
  like_col='like', 
  timestamp_col='timestamp',
  gender_col='decidergender')

Reading data... ✅
Fitting ALS... 
🚀 Preparing data...
✅
Fitting ALS... 
🚀 Preparing data...
🎯 Training male→female ALS...
🎯 Training male→female ALS...


100%|██████████| 15/15 [00:00<00:00, 15.08it/s]



🎯 Training female→male ALS...


100%|██████████| 15/15 [00:00<00:00, 244.14it/s]



🔄 Converting factors to CuPy arrays...
✅ Trained M2F ALS with 33173 males × 33358 females
✅ Trained F2M ALS with 10882 females × 44241 males
Complete! ✅
✅ Trained M2F ALS with 33173 males × 33358 females
✅ Trained F2M ALS with 10882 females × 44241 males
Complete! ✅


In [5]:
engine.run_engagement()

User DF updated ✅


In [6]:
engine.run_elo()

Computing ELO ratings... User DF updated ✅
User DF updated ✅


In [7]:
engine.build_recommender()

Building FAISS recommender... ✅
✅


In [9]:
user_df = engine.user_df
interaction_df = engine.interaction_df
als_model = engine.als_model

In [13]:
import plotly.express as px

In [21]:
px.box(x=user_df[(user_df['gender'] == 'F')&(user_df['is_stable']==True)].elo_rating.dropna())

In [22]:
px.box(x=user_df[(user_df['gender'] == 'M')&(user_df['is_stable']==True)].elo_rating.dropna())

In [29]:
t= user_df[user_df['gender'] == 'F']
px.scatter(t, x=t.elo_rating, color=t['league'])

In [31]:
t= user_df[(user_df['gender'] == 'M')&(user_df['is_stable']==True)]
px.scatter(t, x=t.elo_rating, color=t['league'])

---

In [ ]:
# Test batch performance
import time
t = user_df.dropna()
t = t[t.gender=='M'].sample(20_000).user_id.to_arrow().to_pylist()

start = time.time()
results = engine.recommend_batch(t, k=200)
elapsed = time.time() - start

print(f"✅ Batch recommended for {len(t)} users in {elapsed:.2f} seconds")
print(f"   ({elapsed*1000/len(t):.1f} ms per user)")
print(f"   Sample result for user {t[0]}: {len(results[t[0]])} recommendations")